# 3D-SOCS 3D tracking pipeline demo

This jupyter notebook runs through a few important functions to run the 3D tracking pipeline. For the purpose of this notebook, we will use the official sample dataset, which can be downloaded [here](https://doi.org/10.17617/3.ZQMOJ3). This notebook only goes through key steps with the tracking pipeline, assuming you have already collected frame-synchronized videos from 3D-SOCS. For more infomration on hardware and software requirements for setting up 3D-SOCS, please refer to the raspi instructions in the [github repository](https://github.com/alexhang212/3D-SOCS/tree/main/3DSOCS_raspi)

## Calibration

The first step is multi-view calibration. You will need to collect calibration sequences of a charuco board, moving through the scene. We first need to import required packages:

In [1]:
import sys
import cv2
from cv2 import aruco

# Add the current directory to the system path
sys.path.append("./")

from Calibration import ExtrinsicCalibration
from Calibration import IntrinsicCalibration
from glob import glob
import os
import pickle


In [2]:
## you need to define the path to the videos and the path to a data folder where calibration data will be stored

InputDir = "../SampleDataset/Calibration/videos"
DataDir = "../SampleDataset/Calibration/data"
CamNames = ["mill1","mill2","mill3","mill4","mill5","mill6"]

##this creates the data directory if it does not exist
if not os.path.exists(DataDir):
    os.mkdir(DataDir)

In [3]:
# Definition of calibration board parameters used in the experiment

CalibBoardDict = {
    "widthNum": 5,  # Number of squares in width
    "lengthNum" : 8,  # Number of squares in length
    "squareLen" : 24,  # Length of a square in mm
    "arucoLen" : 19,  # Length of an ArUco marker in mm
    "ArucoDict" : aruco.getPredefinedDictionary(aruco.DICT_4X4_50)  # Predefined ArUco dictionary
}

In [4]:
# Then we can first do intrinsics calibration
VideoPaths = sorted(glob(os.path.join(InputDir, "*.mp4")))

# Intrinsic calibration for each camera
for x in range(len(CamNames)):
    InputVid = VideoPaths[x]
    # Perform intrinsic calibration
    retCal, cameraMatrix, distCoeffs, rvecs, tvecs, pVErr, NewCamMat, roi = IntrinsicCalibration.calibrate_auto(
        InputVid, CalibBoardDict, Freq=10, debug=False
    )
    # Save intrinsic calibration results
    IntSavePath = os.path.join(DataDir, "%s_Intrinsic.p" % CamNames[x])
    pickle.dump((retCal, cameraMatrix, distCoeffs, rvecs, tvecs, pVErr, NewCamMat, roi), open(IntSavePath, "wb"))
    

100%|██████████| 3489/3489 [00:15<00:00, 220.95it/s]


Running optimization algorithm to improve calibration
Images Removed: 1
New Error is 0.892753
Final Error: 0.892753


100%|██████████| 3489/3489 [00:05<00:00, 583.78it/s]


Final Error: 0.561220


100%|██████████| 3489/3489 [00:07<00:00, 490.09it/s]


...

Final Error: 0.643240


100%|██████████| 3489/3489 [00:07<00:00, 462.27it/s]


Final Error: 0.457875


100%|██████████| 3489/3489 [00:06<00:00, 575.46it/s]


Final Error: 0.441298


This will output intrinsic calibration paramters, in terms of the 3x3 intrinsic matrix and the 5x1 distortion matrix

In [7]:
##Load example from camera 1:
pickle.load(open(os.path.join(DataDir, "mill1_Intrinsic.p"), "rb"))

##Intrsinsic camera matrix
print("Camera Matrix")
print(cameraMatrix)

##distortion coefficients
print("Distortion Coefficients")
print(distCoeffs)

Camera Matrix
[[1.42374704e+03 0.00000000e+00 6.47493853e+02]
 [0.00000000e+00 1.42190035e+03 3.70208574e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Coefficients
[[-9.91312998e-03  4.15260744e-01  2.61386448e-03 -2.33446376e-04
  -1.24539145e+00]]


In [6]:
# Next is extrinsics calibration
ExtrinsicCalibration.AutoExtrinsicCalibrator(
    VideoPaths, DataDir, None, CalibBoardDict, CamNames, None, PrimaryCamera=0, Undistort=True
)

# Get reprojection errors for extrinsic calibration
ExtrinsicCalibration.GetReprojectErrors(CamNames, DataDir, VideoName=None)


Computing reprojection error....: 100%|██████████| 1650/1650 [00:09<00:00, 174.40it/s]


Mean Error across views: 1.661813532405055


Computing reprojection error....:   6%|▌         | 96/1650 [00:00<00:09, 166.61it/s]/home/alexchan/Documents/3D-SOCS/3DTracking/Utils/BundleAdjustmentTool.py:188: RuntimeWarning: overflow encountered in square
  EucError = np.sqrt(PointDiffs[:,0]**2 + PointDiffs[:,1]**2)
Computing reprojection error....: 100%|██████████| 1650/1650 [00:15<00:00, 106.44it/s]


Mean Error across views: 330.2637215079684


Computing reprojection error....: 100%|██████████| 1650/1650 [00:14<00:00, 110.32it/s]


...

Computing reprojection error....: 100%|██████████| 1650/1650 [00:12<00:00, 130.49it/s]


Mean Error across views: 75.062853170711
Camera mill1 is best!


Computing reprojection error....: 100%|██████████| 1650/1650 [00:09<00:00, 172.88it/s]

Mean Error across views: 1.661813532405055


1.661813532405055

The extrinsic calibration runs automatically, and tries to find the relative position between each camera. The resulting parameters are a rotation (R) and translation (T) parameter for each camera.

In [10]:
#Load example from camera 2:
R, T = pickle.load(open(os.path.join(DataDir, "mill2_Initial_Extrinsics.p"), "rb"))
            
##Rotation matrix
print("Rotation Matrix")
print(R)

##Translation matrix
print("Translation Matrix")
print(T)

Rotation Matrix
[[ 0.9068442  -0.39809989  0.13838381]
 [ 0.08593723  0.49609628  0.86400421]
 [-0.41261167 -0.77162488  0.48409364]]
Translation Matrix
[[-253.66514532]
 [-682.21246857]
 [ 229.91136269]]


## 3D Tracking

Now that the calibration is done, the parameters are automatically saved in the data folder, and you can now do 3D tracking!

In [11]:
## Import stuff
import os
import sys
import pickle
import pandas as pd
sys.path.append("../Repositories/DeepLabCut-live")
sys.path.append("./")

from ultralytics import YOLO
from dlclive import DLCLive, Processor
import deeplabcut as dlc
from Utils import Track3DFunctions as T3D

from natsort import natsorted

import datetime as dt
import numpy as np

import cv2
from tqdm import tqdm

/home/alexchan/anaconda3/envs/tf/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Loading DLC 2.3.3...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [12]:
## We have to define a list of keypoints, and the order it is outputted by the model
INFERENCE_BODYPARTS = [
    "hd_bill_tip",
    "hd_bill_base",
    "hd_cheek_left",
    "hd_cheek_right",
    "hd_head_back",
    "bd_tail_base",
    "bd_bar_left_front",
    "bd_bar_left_back",
    "bd_bar_right_back",
    "bd_bar_right_front",
    "hd_eye_right",
    "hd_eye_left",
    "bd_shoulder_right",
    "bd_tail_tip",
    "hd_neck_right",
    "bd_shoulder_left",
    "hd_neck_left",
    "bd_keel_top"
]


In [13]:
#Define more inputs

###Base video directory
InputDir = "../SampleDataset/3DTracking/bird_3B00186CDA_trial_32_time_2023-11-05 10_30_28.117871/"

###Where the calibration data is
DataDir =  "../SampleDataset/Calibration/data"

##Define data directory to store tracking info, and where the videos are
SubDataDir = os.path.join(InputDir, "data")
VideoDir = os.path.join(InputDir, "videos")

#Camera names
CamNames = ["mill1", "mill2", "mill3", "mill4", "mill5", "mill6"]

##Path to the trained YOLO and DLC models
YOLOPath = "../SampleDataset/Weights/GretiBlutiYOLO.pt"
ExportModelPath = "../SampleDataset/Weights/Greti_DLC"


In [14]:
## First run YOLO for object detection!
VideoFiles = natsorted([os.path.join(VideoDir, subfile) for subfile in os.listdir(VideoDir) if subfile.endswith(".mp4")])


OutbboxDict, BBoxConfDict = T3D.RunYOLOTrack(YOLOPath, VideoFiles, DataDir, CamNames, startFrame=0, TotalFrames=-1, ScaleBBox=1, 
                                                FrameDiffs=[0]*len(CamNames), Objects=["greti", "bluti"], Extrinsic="Initial", undistort=True, YOLOThreshold=0.7)

##Saving the data
pickle.dump(OutbboxDict, open(os.path.join(SubDataDir, "OutbboxDict.p"), "wb"))
pickle.dump(BBoxConfDict, open(os.path.join(SubDataDir, "BBoxConfDict.p"), "wb"))

Running YOLO:: 100%|██████████| 138/138 [01:13<00:00,  1.87it/s]


The output of the YOLO function are 2 dictionaries, one stores the bounding box outputs, the other stores the confidences.

The dictionary is made of nested dictionaries structured as follows:
```
{"frame": {
    "CamName":{
    "TrackingID":[x1,y1,x2,y2]
    ....
}}}
```
First level is the frame number, then camera names, then individual bounding boxes for each tracking ID. The bounding box confidences are stored in the same way

In [16]:
##Example for frame 0
OutbboxDict = pickle.load(open(os.path.join(SubDataDir, "OutbboxDict.p"), "rb"))
OutbboxDict[0]

{'mill1': {'mill1-greti-1.0': [1295.8067626953125,
   208.7423095703125,
   1493.1690673828125,
   648.7586669921875],
  'mill1-greti-2.0': [1661.8609008789062,
   482.964599609375,
   1919.7001342773438,
   1080.0]},
 'mill2': {'mill2-greti-1.0': [502.1390380859375,
   257.21807861328125,
   681.802001953125,
   499.021484375]},
 'mill3': {'mill3-greti-2.0': [376.87371826171875,
   94.2266845703125,
   560.5238647460938,
   303.81610107421875]},
 'mill4': {'mill4-greti-1.0': [80.33905029296875,
   301.81353759765625,
   607.6127319335938,
   553.2652587890625]},
 'mill5': {},
 'mill6': {'mill6-greti-1.0': [623.7623291015625,
   359.3936767578125,
   828.4041748046875,
   547.9381103515625]}}

Next, we run the DeepLabCut model to get 2D postures.

In [19]:
##Prepare deeplabcut inference model
dlc_proc = Processor()
dlc_liveObj = DLCLive(ExportModelPath, processor=dlc_proc)


OutbboxDict = pickle.load(open(os.path.join(SubDataDir, "OutbboxDict.p"), "rb"))
BBoxConfDict = pickle.load(open(os.path.join(SubDataDir, "BBoxConfDict.p"), "rb"))

# Run DeepLabCut to track body parts in the detected objects
Out2DDict = T3D.RunDLC(dlc_liveObj, OutbboxDict, BBoxConfDict, DataDir, 
                        VideoFiles, CamNames, CropSize=(320,320), INFERENCE_BODYPARTS=INFERENCE_BODYPARTS,
                        startFrame=0, TotalFrames=-1, FrameDiffs=[0]*len(CamNames), Extrinsic="Initial",
                        DLCConfidenceThresh=0.3, YOLOThreshold=0.5,
                        undistort=True)

# Save DeepLabCut output
pickle.dump(Out2DDict, open(os.path.join(SubDataDir, "Out2DDict.p"), "wb"))

Running DLC:: 100%|██████████| 138/138 [01:13<00:00,  1.88it/s]


The output of the 2D keypoint detection step is similar to YOLO, also nested dictionaries.

Here is the structure:
```
{"frame": {
    "CamName":{
    "TrackingID":{
        "ID_KeypointName":[x,y]
    }
    ....
}}}
```

First level is frame number, then camera names, then tracking ID, then individual keypoint names.

In [21]:
#Example for frame 0
Out2DDict = pickle.load(open(os.path.join(SubDataDir, "Out2DDict.p"), "rb"))
Out2DDict[0]

{'mill1': {'mill1-greti-1.0': {'mill1-greti-1.0_hd_head_back': [1337.6382026672363,
    571.0718994140625],
   'mill1-greti-1.0_bd_bar_left_back': [1459.9082336425781,
    466.63531494140625],
   'mill1-greti-1.0_bd_bar_right_back': [1393.0919799804688,
    475.8860778808594],
   'mill1-greti-1.0_bd_bar_right_front': [1357.7661628723145,
    497.9913024902344]}},
 'mill2': {'mill2-greti-1.0': {'mill2-greti-1.0_hd_bill_base': [520.6679267883301,
    479.22900390625],
   'mill2-greti-1.0_hd_cheek_left': [569.3762283325195, 447.8141174316406],
   'mill2-greti-1.0_hd_head_back': [545.3655242919922, 419.7465362548828],
   'mill2-greti-1.0_bd_tail_base': [628.4700775146484, 307.0471954345703],
   'mill2-greti-1.0_bd_bar_left_front': [632.3062591552734,
    386.95167541503906],
   'mill2-greti-1.0_bd_bar_left_back': [623.7471771240234, 351.99369049072266],
   'mill2-greti-1.0_hd_eye_left': [544.5481491088867, 469.30853271484375]}},
 'mill3': {'mill3-greti-2.0': {'mill3-greti-2.0_bd_tail_base'

After getting 2D keypoint estimates, we have to triangulate them into 3D coordinates. Before we do this, we also quickly run a 2D rolling average filter to make the detection less jittery.

In [23]:
##Load outputs from inference
Out2DDict = pickle.load(open(os.path.join(SubDataDir, "Out2DDict.p"), "rb"))
BBoxConfDict = pickle.load(open(os.path.join(SubDataDir, "BBoxConfDict.p"), "rb"))

Filtered2DDict = T3D.Filter2D(Out2DDict, CamNames, INFERENCE_BODYPARTS, WindowSize=3)
pickle.dump(Filtered2DDict, open(os.path.join(SubDataDir, "Filtered2DDict.p"), "wb"))


##Do matching and triangulation
Out3DDict, Out2DDict = T3D.TriangulateFromPointsMulti(Filtered2DDict, BBoxConfDict, DataDir, 
                                                        CamNames, INFERENCE_BODYPARTS, Extrinsic="Initial",
                                                        TotalFrames=-1, YOLOThreshold=0.5, CombinationThreshold=0.7,
                                                        DminThresh=50)

pickle.dump(Out3DDict, open(os.path.join(SubDataDir, "Out3DDict.p"), "wb"))


Doing Matching...: 100%|██████████| 137/137 [00:02<00:00, 48.85it/s]


[{'mill4-greti-94.0', 'mill5-greti-96.0', 'mill2-greti-86.0', 'mill1-greti-25.0', 'mill5-greti-22.0', 'mill2-greti-26.0', 'mill4-greti-20.0', 'mill6-greti-23.0', 'mill1-greti-2.0', 'mill4-bluti-20.0', 'mill3-greti-28.0'}, {'mill2-greti-1.0', 'mill1-greti-1.0', 'mill6-greti-1.0', 'mill4-greti-1.0', 'mill5-greti-4.0', 'mill3-greti-2.0'}, {'mill3-bluti-2.0', 'mill1-bluti-64.0', 'mill6-greti-53.0', 'mill2-bluti-60.0', 'mill5-bluti-4.0', 'mill6-bluti-53.0', 'mill2-greti-100.0', 'mill4-bluti-1.0'}]


Triangulating...: 100%|██████████| 137/137 [00:02<00:00, 55.10it/s]


3D keypoint data is also stored in a similar way, in nested dictionaries.

Here is the structure:
```
{"frame": {
        "ID_KeypointName":[x,y,z]
    ....
}}
```

First level is frame number, then keypoints names, with the first string before the first underscore ("_") as the ID of the bird

In [24]:
##Example for frame 0
Out3DDict = pickle.load(open(os.path.join(SubDataDir, "Out3DDict.p"), "rb"))
Out3DDict[0]

{'1_bd_bar_left_back': array([      147.2,     -30.446,      770.29]),
 '1_bd_bar_left_front': array([     152.43,     -22.492,      782.09]),
 '1_bd_bar_right_back': array([     125.27,      -28.64,      763.35]),
 '1_bd_bar_right_front': array([     116.83,     -23.613,      773.84]),
 '1_bd_tail_base': array([        134,     -59.356,      768.34])}

# Post Processing

Finally is post processing, there are multiple steps involved, and is described in detail in the manuscript. 

In [26]:
##Import stuff
import os
import pickle
import numpy as np
import pandas as pd

import sys
sys.path.append("Utils/")
import PostProcessFunctions as PP

#Define stuff
InputDir = "../SampleDataset/3DTracking/bird_3B00186CDA_trial_32_time_2023-11-05 10_30_28.117871/"
DataDir = "../SampleDataset/Calibration/data"
SubDataDir = os.path.join(InputDir, "data")

# Define camera names and their respective image sizes
CamNames = ["mill1", "mill2", "mill3", "mill4", "mill5", "mill6"]
imsizeDict = {
    "mill1": (1920, 1080), "mill2": (1280, 720), "mill3": (1280, 720),
    "mill4": (1280, 720), "mill5": (1280, 720), "mill6": (1280, 720)
}

# Load inference files
Out3DDict = pickle.load(open(os.path.join(SubDataDir, "Out3DDict.p"), "rb"))
Out2DDict = pickle.load(open(os.path.join(SubDataDir, "Filtered2DDict.p"), "rb"))



In [29]:
## First filter: reproject points and filter if the reprojection is outside of the screen
FilterReprojectOut = PP.FilterbyReproject(Out3DDict, DataDir, CamNames, imsizeDict, Ratio=0.5, Extrinsic="Initial")

## Second filter: filter by distance to mean head and body

# Define head and body points for filtering
HeadPoints = ["hd_eye_right", "hd_eye_left", "hd_cheek_left", "hd_cheek_right", "hd_bill_tip", "hd_bill_base", "hd_neck_left", "hd_neck_right"]
BodyPoints = ["bd_bar_left_front", "bd_bar_left_back", "bd_bar_right_front", "bd_bar_right_back", "bd_tail_base", "bd_keel_top", "bd_tail_tip", "bd_shoulder_left", "bd_shoulder_right"]

# Filter points by head and body
Filtered_3DDict = PP.FilterPoints(FilterReprojectOut, HeadPoints)
Filtered_3DDict = PP.FilterPoints(Filtered_3DDict, BodyPoints)

## Interpolate data
Linear3DDict = PP.InterpolateData(Filtered_3DDict, Type="pchip", interval=1, show=False)
pickle.dump(Linear3DDict, open(os.path.join(SubDataDir, "MultiFiltered_Out3DDict.p"), "wb"))


In [31]:
## Next, we use the median species skull and "slot in" the head based on the rotation of head keypoints
Filtered_3DDict = pickle.load(open(os.path.join(SubDataDir, "MultiFiltered_Out3DDict.p"), "rb"))

##First save the data, as this function does this per individual
pickle.dump(Linear3DDict, open(os.path.join(SubDataDir, "MultiObjectFiltered_Out3DDict.p"), "wb"))

##Find all unique IDs
Out3DDictKeys = [list(subDict.keys()) for subDict in Linear3DDict.values()]
Out3DDictKeysFlat = []
[Out3DDictKeysFlat.extend(x) for x in Out3DDictKeys]
UnqIDs = sorted(list(set([x.split("_")[0] for x in list(set(Out3DDictKeysFlat))])))
SpeciesObjectDict = pickle.load(open(os.path.join(InputDir, "MedianSpeciesObjects.p"), "rb"))  # Median skeleton for species!

# Filter by object for each individual ID
for IndID in UnqIDs:
    Linear3DDict = pickle.load(open(os.path.join(SubDataDir, "MultiObjectFiltered_Out3DDict.p"), "rb"))

    HeadFilteredDictRotation = PP.FilterbyObject(Linear3DDict, SpeciesObjectDict["GRETI"]["hd"], HeadPoints, IndID, ObjFilterThresh=50)
    BodyFilteredDictRotation = PP.FilterbyObject(HeadFilteredDictRotation, SpeciesObjectDict["GRETI"]["bd"], BodyPoints, IndID, ObjFilterThresh=50)

    # Save the filtered data
    pickle.dump(BodyFilteredDictRotation, open(os.path.join(SubDataDir, "MultiObjectFiltered_Out3DDict.p"), "wb"))

# Final interpolation of the filtered data
FinalOut3DDict = PP.InterpolateData(BodyFilteredDictRotation, Type="pchip", interval=1, show=False)

# Save the final processed data
pickle.dump(FinalOut3DDict, open(os.path.join(SubDataDir, "Out3DDictProcessed.p"), "wb"))
print("Processing done!")


Processing done!


# Visualization

Finally, you can visualize the data with overlayed visual fields.

In [32]:
#Import and define stuff

import os
import sys
import pickle
import numpy as np
from natsort import natsorted
sys.path.append("Utils")
from ReprojectVisualFields import Visualize3D_VS

# Dictionary to map body parts to their respective colors for visualization
ColourDictionary = {
    "hd_bill_tip": (180, 95, 26),
    "hd_bill_base":(92, 228, 248),
    "hd_eye_left" :(137, 227, 87),
    "hd_eye_right":(59, 51, 237),
    "hd_cheek_left":(241, 193, 153),
    "hd_cheek_right":(0, 120, 255),
    "hd_neck_left": (156, 61, 129),
    "hd_neck_right":(17, 194, 245),
    "hd_head_back":(90, 131, 181),
    "bd_bar_left_front":(216, 113, 28),
    "bd_bar_left_back":(126, 194, 46),
    "bd_bar_right_front":(0, 120, 255),
    "bd_bar_right_back":(81, 97, 246),
    "bd_tail_base":  (68, 106, 152)
}


In [33]:
##Define parameters
# Input directory containing the dataset
InputDir = "../SampleDataset/3DTracking/bird_3B00186CDA_trial_32_time_2023-11-05 10_30_28.117871/"
# Directory containing calibration data
DataDir = "../SampleDataset/Calibration/data"
# Subdirectory containing the processed data
SubDataDir = os.path.join(InputDir,"data")
# Directory containing the video files
VideoDir = os.path.join(InputDir,"videos")

# List of camera names
CamNames = ["mill1","mill2","mill3","mill4","mill5","mill6"]

# Get sorted list of video files
VideoFiles = natsorted([os.path.join(VideoDir,subfile) for subfile in os.listdir(VideoDir) if subfile.endswith(".mp4")])
# Load the processed 3D data dictionary
Out3DDict = pickle.load(open(os.path.join(SubDataDir,"Out3DDictProcessed.p"),"rb"))

In [34]:
# Visualize the 3D tracking data
Visualize3D_VS(Out3DDict, DataDir, VideoFiles, CamNames, FrameDiffs=[0]*len(CamNames),
                startFrame=0, TotalFrames=-1, VisualizeIndex=2, ColourDictionary=ColourDictionary,
                Extrinsic="Initial", show=False, save=True, VSAngle=60, ElvAngle=0, Magnitude=3)

100%|██████████| 138/138 [00:02<00:00, 49.15it/s]


{}

This saves an output video in the current directory, you can take a look at the tracking results!